In [2]:
#Step 1: Install Required Libraries
pip install transformers datasets pandas scikit-learn torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [5]:
#Step 2: Load Dataset
import pandas as pd

# Replace with your dataset file name
df = pd.read_csv("/content/dataset-tickets-multi-lang-4-20k.csv", engine='python')

# Combine 'subject' and 'body' columns to create the 'text' column
df['text'] = df['subject'].fillna('') + ' ' + df['body'].fillna('')

# Use the 'type' column as the 'label'
df = df[['text', 'type']]
df = df.rename(columns={'type': 'label'})

df = df.dropna()
print(df.head())

                                                text     label
0  Unvorhergesehener Absturz der Datenanalyse-Pla...  Incident
1  Customer Support Inquiry Seeking information o...   Request
2  Data Analytics for Investment I am contacting ...   Request
3  Krankenhaus-Dienstleistung-Problem Ein Medien-...  Incident
4  Security Dear Customer Support, I am reaching ...   Request


In [6]:
#step 3: Define Candidate Labels
candidate_labels = [
    "Billing Issue",
    "Login Problem",
    "Technical Error",
    "Account Management",
    "Subscription",
    "Password Reset",
    "App Crash",
    "Feature Request"
]


In [7]:
#Step 4: Zero-Shot Classification using Hugging Face Pipeline
from transformers import pipeline

# Load the zero-shot classifier
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

def get_top_3_tags(text, labels):
    result = classifier(text, labels, multi_label=True)
    top_3 = list(zip(result['labels'], result['scores']))[:3]
    return top_3

# Test with one ticket
text = df.iloc[0]['text']
print("Ticket:", text)
print("Top 3 Tags:", get_top_3_tags(text, candidate_labels))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


Ticket: Unvorhergesehener Absturz der Datenanalyse-Plattform Die Datenanalyse-Plattform brach unerwartet ab, da die Speicheroberfläche zu gering war. Ich habe versucht, Laravel 8 und meinen MacBook Pro neu zu starten, aber das Problem behält sich bei. Ich benötige Ihre Unterstützung, um diesen Fehler zu beheben.
Top 3 Tags: [('Technical Error', 0.4011070728302002), ('Login Problem', 0.3523354232311249), ('Feature Request', 0.163441002368927)]


In [8]:
#Step 5: Few-Shot Prompting (Optional - GPT-based models)
few_shot_examples = [
    ("I can't log in to my account", ["Login Problem"]),
    ("I was charged after canceling", ["Billing Issue", "Subscription"]),
    ("The app crashes on startup", ["App Crash", "Technical Error"])
]

prompt = ""
for ex in few_shot_examples:
    prompt += f"Ticket: {ex[0]}\nTags: {', '.join(ex[1])}\n"
prompt += f"Ticket: {df.iloc[1]['text']}\nTags:"

# Use OpenAI or similar LLM to complete the prompt (if allowed)
print(prompt)


Ticket: I can't log in to my account
Tags: Login Problem
Ticket: I was charged after canceling
Tags: Billing Issue, Subscription
Ticket: The app crashes on startup
Tags: App Crash, Technical Error
Ticket: Customer Support Inquiry Seeking information on digital strategies that can aid in brand growth and details on the available services. Looking forward to learning more to help our business grow. Thank you, and I look forward to hearing from you soon.
Tags:
